
# Demo completa — MonteCarlo + Portfolio (con datos simulados)

Este notebook prueba de extremo a extremo tus módulos:

- `models.py` (incluye `PriceSeries` y `Portfolio`)
- `montecarlo.py` (`MonteCarloSimulation` genérico para PriceSeries/Portfolio)
- `reports.py` (`MonteCarloPlots` y `MonteCarloReport` con seaborn)
- `utils.py` (helpers de limpieza y métricas)
- `providers.py` (se muestra cómo se usaría, pero **no** descargamos datos reales)

> **Nota:** la demo usa **datos simulados** (GBM) para que funcione offline.



## 1) Preparación de entorno
Añadimos la carpeta de trabajo al `sys.path` y cargamos módulos.


In [ ]:

import sys, os
from pathlib import Path
BASE = Path('/mnt/data')
sys.path.insert(0, str(BASE))

print("Base path:", BASE)
print("Listado:", [p.name for p in BASE.iterdir()])


In [ ]:

# Imports del proyecto
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from models import PriceSeries, Portfolio
from utils import to_business_days, clean_price_frame, log_returns, drawdowns, var_cvar
from montecarlo import MonteCarloSimulation
from reports import MonteCarloPlots, MonteCarloReport

import seaborn as sns
sns.set_theme(context="notebook", style="whitegrid")
print("Módulos importados OK.")



## 2) Generar datos simulados (dos activos)
Creamos dos series de precios de 3 años laborables con parámetros distintos y las envolvemos en `PriceSeries`.


In [ ]:

# Fechas laborables (3 años aprox.)
idx = pd.bdate_range(start='2022-01-03', end='2024-12-31')

# Función para simular GBM
def gbm_prices(n, mu=0.10, sigma=0.20, s0=100.0, seed=0):
    rng = np.random.default_rng(seed)
    dt = 1/252
    z = rng.standard_normal(n-1)
    log_increments = (mu - 0.5*sigma**2)*dt + sigma*np.sqrt(dt)*z
    log_path = np.r_[0.0, np.cumsum(log_increments)]
    return s0 * np.exp(log_path)

p1 = gbm_prices(len(idx), mu=0.12, sigma=0.25, s0=100, seed=1)
p2 = gbm_prices(len(idx), mu=0.06, sigma=0.12, s0=50,  seed=2)

df1 = pd.DataFrame({"price": p1}, index=idx)
df2 = pd.DataFrame({"price": p2}, index=idx)

ps1 = PriceSeries(symbol="ASSET_A", asset_type="equity", currency="USD", provider="sim", data=df1)
ps2 = PriceSeries(symbol="ASSET_B", asset_type="equity", currency="USD", provider="sim", data=df2)

ps1.data.head(), ps2.data.head()



## 3) Construir `Portfolio` (60% ASSET_A, 40% ASSET_B)


In [ ]:

port = Portfolio(positions=[ps1, ps2], weights=[0.6, 0.4], name="Demo 60_40", currency="USD")
aligned = port.aligned_prices()
print("Aligned prices shape:", aligned.shape)
aligned.head()



## 4) Serie de valor de la cartera y drawdowns


In [ ]:

eq = port.value_series(initial_capital=1.0)
dd = drawdowns(eq)
display(eq.head())
display(dd.head())

fig, ax = plt.subplots(figsize=(10,3.5))
eq.plot(ax=ax, title=f"Portfolio Equity — {port.name}")
plt.show()

fig, ax = plt.subplots(figsize=(10,3.5))
dd.plot(ax=ax, title=f"Portfolio Drawdown — {port.name}")
plt.show()



## 5) MonteCarlo sobre **PriceSeries** (ASSET_A)


In [ ]:

mc_a = MonteCarloSimulation(price_series=ps1)  # acepta PriceSeries
summ_a = mc_a.simulate_and_summarize(days=252, n_sims=500, seed=42)
list(summ_a.keys()), summ_a['prices'].shape



### Plots (histórico / simulaciones / combinado / histograma final) para **ASSET_A**


In [ ]:

plots_a = MonteCarloPlots(mc_a)
_ = plots_a.plot_history()
_ = plots_a.plot_simulations(summ_a["prices"])
_ = plots_a.plot_history_with_simulations(summ_a["prices"], currency_symbol="$")
_ = plots_a.plot_final_hist(summ_a["prices"], currency_symbol="$")
plt.show()



## 6) MonteCarlo sobre **Portfolio**
La clase `MonteCarloSimulation` también funciona con `Portfolio` gracias a `to_price_dataframe()`.


In [ ]:

mc_p = MonteCarloSimulation(price_series=port)  # acepta Portfolio
summ_p = mc_p.simulate_and_summarize(days=252, n_sims=500, seed=7)
list(summ_p.keys()), summ_p['prices'].shape



### Plots (histórico / simulaciones / combinado / histograma final) para **Portfolio**


In [ ]:

plots_p = MonteCarloPlots(mc_p)
_ = plots_p.plot_history()
_ = plots_p.plot_simulations(summ_p["prices"])
_ = plots_p.plot_history_with_simulations(summ_p["prices"], currency_symbol="$")
_ = plots_p.plot_final_hist(summ_p["prices"], currency_symbol="$")
plt.show()



## 7) Exportar informe PDF (histórico, combinado y distribución final)


In [ ]:

from pathlib import Path
outdir = Path('/mnt/data/outputs'); outdir.mkdir(exist_ok=True)
pdf_path_a = outdir/'report_asset_A.pdf'
pdf_path_p = outdir/'report_portfolio.pdf'

rep_a = MonteCarloReport(mc_a)
rep_p = MonteCarloReport(mc_p)

rep_a.to_pdf(summ_a["prices"], str(pdf_path_a))
rep_p.to_pdf(summ_p["prices"], str(pdf_path_p))

print("PDFs generados:")
print(pdf_path_a)
print(pdf_path_p)



## 8) (Opcional) Proveedores `yfinance` / `alpha_vantage`
**No se ejecuta descarga real** en esta demo. Ejemplo de uso:


In [ ]:

from providers import YahooProvider, AlphaVantageProvider

print("Demostración de interfaz (sin ejecutar descargas en offline):")
print("YahooProvider().price_history(symbols=['AAPL','MSFT'], periods=60)  # devolvería un dict de DataFrames")
print("AlphaVantageProvider(api_key='TU_API').price_history(symbols='AAPL', periods=60)  # idem")
